In [1]:
# import numpy as np
import pandas as pd
import math
import os
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 200)

In [2]:
# context = pd.read_csv('..//data/preprocessed//context_1217_1230.csv')
context = pd.read_csv('/home/jovyan/df-smart-channel/graph/data/preprocessed/context_1217_0131.csv')

In [16]:
context_click = context[context.click > 0]

In [24]:
le = LabelEncoder()
context_click.cust_no = le.fit_transform(context_click.cust_no)
context_click.item_id = le.fit_transform(context_click.item_id)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [25]:
user_list = context_click.cust_no.unique()
item_list = context_click.item_id.unique()

## Splitting

In [27]:
train_start = 20211217
train_end = 20211225
test_end = 20211226

In [33]:
def splitting(user_list, train_start, test_date):
    context_train = context_click[(context_click.date >= train_start) & (context_click.date < test_date)][['cust_no', 'item_id', 'show', 'click']]
    context_test = context_click[context_click.date == test_date][['cust_no', 'item_id', 'show', 'click']]
    
    cust_item_pair_train = context_train.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()
    cust_item_pair_test = context_test.groupby(['cust_no', 'item_id'])['click'].sum().reset_index()
    
    # cust_item_pair_train = cust_item_pair_train[cust_item_pair_train.click > 0]
    # cust_item_pair_test = cust_item_pair_test[cust_item_pair_test.click > 0]
    
    trainUniqueUsers = set(cust_item_pair_train.cust_no.unique())
    # testUniqueUsers = set(cust_item_pair_test.cust_no.unique())
    cust_item_pair_test = cust_item_pair_test[cust_item_pair_test.cust_no.isin(trainUniqueUsers)]
    
    items_by_cust_train = cust_item_pair_train.groupby('cust_no')['item_id'].unique()
    items_by_cust_test = cust_item_pair_test.groupby('cust_no')['item_id'].unique()
    
    cust_items_dict_train = dict(items_by_cust_train.apply(list))
    cust_items_dict_test = dict(items_by_cust_test.apply(list))
    return cust_items_dict_train, cust_items_dict_test

In [36]:
def export_file(date, pairs_dict_train, pairs_dict_test):
    dir_name = f'date={date}'
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
        
    ## train
    file_path = os.path.join(dir_name, 'train.txt')
    file = open(file_path,"w")

    for key,value in pairs_dict_train.items():
        file.write('%s' % key)
        # for item in value:
        #     file.write('%s ' % item)
        for i, item in enumerate(value):
            if math.isnan(item):
                continue
            else:
                if i == (len(value)-1):
                    file.write(' %s' % int(item))
                else:
                    file.write(' %s' % int(item))
        file.write('\n')
    file.close()
    ## test
    file_path = os.path.join(dir_name, 'test.txt')
    file = open(file_path,"w")

    for key,value in pairs_dict_test.items():
        file.write('%s' % key)
        # for item in value:
        #     file.write('%s ' % item)
        for i, item in enumerate(value):
            if math.isnan(item):
                continue
            else:
                if i == (len(value)-1):
                    file.write(' %s' % int(item))
                    file.write('\n')
                else:
                    file.write(' %s' % int(item))
    file.close()

In [37]:
for test_date in range(20211226, 20211231+1):
    cust_items_dict_train, cust_items_dict_test = splitting(user_list = user_list, train_start = 20211217, test_date = test_date) 
    export_file(date = str(test_date), pairs_dict_train = cust_items_dict_train, pairs_dict_test = cust_items_dict_test )

In [35]:
for test_date in range(20211226, 20211231+1):
    cust_items_dict_train, cust_items_dict_test = splitting(user_list = user_list, train_start = 20211217, test_date = test_date) 
    len_tr = len(list(cust_items_dict_train.keys()))
    len_te = len(list(cust_items_dict_test.keys()))
    print(len_tr)
    print(len_te)
    print('---')

8852
50
---
9606
105
---
10964
124
---
12372
114
---
13693
64
---
14486
47
---


In [12]:
for test_date in range(20211226, 20211231+1):
    cust_items_dict_train, cust_items_dict_test = splitting(user_list = user_list, train_start = 20211217, test_date = test_date) 
    len_tr = len(list(cust_items_dict_train.keys()))
    len_te = len(list(cust_items_dict_test.keys()))
    print(len_tr)
    print(len_te)
    print('---')

8852
50
---
9606
105
---
10964
124
---
12372
114
---
13693
64
---
14486
47
---


In [16]:
for test_date in range(20211226, 20211231+1):
    cust_items_dict_train, cust_items_dict_test = splitting(user_list = user_list, train_start = 20211217, test_date = test_date) 
    len_tr = len(list(cust_items_dict_train.keys()))
    len_te = len(list(cust_items_dict_test.keys()))
    print(len_tr)
    print(len_te)
    print('---')

8852
804
---
9606
1463
---
10964
1532
---
12372
1435
---
13693
857
---
14486
524
---
